In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/kaggle/input/mlkaggle/vuong_train.csv')

In [3]:
# df['label'] = df['label'].replace(-1,0)

In [4]:
df

,Unnamed: 0,len,Comment,Rating
0,0,13,xôi dẻo đồ_ăn đậm vị hộp xôi được lót lá trông...,1.0
1,1,53,gọi ship 1 xuất cari gà bánh naan và 3 miếng g...,0.0
2,2,48,thời_tiết lạnh như này cả nhà rủ nhau đến lega...,1.0
3,3,59,em có đọc review thấy mng bảo trà sữa nướng đề...,0.0
4,4,93,đồ_ăn rất ngon nhà_hàng cũng rất đẹp tất_cả đề...,1.0
...,...,...,...,...
9064,9068,30,thực_sự mà nói thấy mọi người đánh_giá nhiều q...,1.0
9065,9069,134,lẩu thái tômyum đc 19 lò_đúc hbt hn theo đánh_...,1.0
9066,9070,45,ngay từ lúc đầu_tiên bước vào nhà_hàng đã được...,1.0
9067,9071,17,đặt ăn thử mà thấy ngón cá emo rất mê sẽ còn ủ...,1.0


In [5]:
# df = df[['RevId','Comment','label','Rating']]

In [6]:
df['Comment'].isnull().value_counts()

False    9069
Name: Comment, dtype: int64

In [7]:
# df['len'].describe()

In [8]:
# list_len = df['len'].to_list()
# list_text = df['Comment'].to_list()
# list_label = df['label'].to_list()

# for i in range(len(list_len)):
#     if list_len[i] >= 256:
#         print(list_text[i])
#         tmp = list_text[i].split()
#         list_text[i] =  ' '.join(tmp[0:210]) + ' ' + ' '.join(tmp[-46:])
#         print()
#         print(list_text[i])
#         print(list_label[i])
#         print("hihihi")

# df['Comment'] = list_text

In [9]:
df = df.dropna()
df.shape

(9069, 4)

In [10]:
df['Rating'] = df['Rating'].astype(int)

In [11]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=1000, random_state=42)
df_train.shape, df_test.shape

((8069, 4), (1000, 4))

In [12]:
from sklearn.utils import shuffle

df_train_x = df_train[df_train['Rating'] == 1]
df_train_y = df_train[df_train['Rating'] == 0]
print(df_train_x.shape, df_train_y.shape)
df_train_x = df_train_x.sample(5500)
df_train_y= df_train_y.sample(1000)

df_train = pd.concat([df_train_x, df_train[df_train['Rating'] == 0], df_train_y],axis = 0)

df_train = shuffle(df_train)

(6354, 4) (1715, 4)


In [13]:
# df_test = df_test[['Comment','label']]

In [14]:
df_train['Rating'].value_counts()

1    5500
0    2715
Name: Rating, dtype: int64

In [15]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import pandas as pd
import torch
import numpy
import re
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from sklearn.svm import SVC
from joblib import dump
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.dataloader import default_collate
import random
import torch.backends.cudnn as cudnn
import numpy as np

from keras.models import Sequential
from keras.layers import *
from keras import backend as K
from keras import initializers
from keras import regularizers, constraints
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint, EarlyStopping

import joblib

In [16]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

cudnn.benchmark = False
cudnn.deterministic = True

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base").to(device)
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer
print(device)

cuda


In [18]:
v_phobert = AutoModel.from_pretrained("vinai/phobert-base").to(device)
v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
# v_phobert.embeddings.position_embeddings = nn.Embedding(1024, 768, padding_idx=1)

In [20]:
v_phobert

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [21]:
v_tokenizer

PreTrainedTokenizer(name_or_path='vinai/phobert-base', vocab_size=64000, model_max_len=256, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'})

In [22]:
MAX_LEN = 256
print("LOAD phoBERT DONE")

def get_emb_vector(input_ids):
    input_ids  = torch.tensor([input_ids]).to(torch.long)
    with torch.no_grad():
        features = phobert(input_ids.to(torch.device("cuda:0")))
    #print(features)
    emb_vecs = features[0].cpu().numpy()[0]#[1:-1]
    #print(emb_vecs)
    return emb_vecs

LOAD phoBERT DONE


In [23]:
def text2ids(text):
    # print(tokenizer.encode("<pad> nhà <pad>"))
    tkz = tokenizer.encode(text)
    return tkz

## Data Generator

In [24]:
from keras.utils.data_utils import Sequence

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, ids, labels, batch_size=16,  max_seq_len=256, feature_len=768, n_classes=18, shuffle=True):
        self.max_seq_len = max_seq_len
        self.feature_len = feature_len
        self.batch_size = batch_size
        self.labels = labels
        self.ids = ids
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.ids) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.ids))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, idx_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.max_seq_len, self.feature_len))
        y = np.empty((self.batch_size), dtype=int)
        # Generate data
        for i, idx in enumerate(idx_temp):
            X[i,] = get_emb_vector(self.ids[idx])
            # Store class
            y[i] = self.labels[idx]
        # X = X[:,:,:,np.newaxis]
        # y = to_categorical(y, num_classes=self.n_classes)
        # print(y.shape)
        # exit()
        return X, y

In [25]:
# train_generator = DataGenerator(ids_train, y_train, batch_size=8, n_classes=len(classes))
# valid_generator = DataGenerator(ids_test, y_test, batch_size=8, n_classes=len(classes), shuffle=False)

## Finetuning BERT

In [26]:
# def standardize_data(row):
#     # Xóa dấu chấm, phẩy, hỏi ở cuối câu
#     row = re.sub(r"[\.,\?]+$-", "", row)
#     # Xóa tất cả dấu chấm, phẩy, chấm phẩy, chấm thang, ... trong câu
#     row = row.replace(",", " ").replace(".", " ") \
#         .replace(";", " ").replace("“", " ") \
#         .replace(":", " ").replace("”", " ") \
#         .replace('"', " ").replace("'", " ") \
#         .replace("!", " ").replace("?", " ") \
#         .replace("-", " ").replace("?", " ")
#     return row

In [27]:
from tqdm import tqdm

class Bert_fine_tuning(nn.Module):
    def __init__(self):
        super(Bert_fine_tuning, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base").to(device)
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(in_features=768, out_features = 500, bias=True).to(device),
            nn.BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True).to(device),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=500, out_features = 2, bias=True).to(device)
        )
        self.criterior = nn.CrossEntropyLoss()
        # self.criterior = nn.BCEWithLogitsLoss()
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        param_optimizer = list(self.bert.named_parameters())
        self.optimizer =optim.AdamW(
#             [{'params': self.bert.parameters(), 'lr': 8e-5}, 
#             {'params': self.classifier.parameters(), 'lr': 1e-3}],
            [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
            {'params': self.classifier.parameters(), 'lr': 1e-3}],
            lr = 2e-5
#              momentum=0.9
        )
#         self.sigmoid = nn.Sigmoid()
      
    
    def tokenize(self,v_text, label):
        v_tokenized = []
        max_len = 256 
        for i_text in v_text:
#             i_text = standardize_data(i_text)
            line = self.tokenizer.encode(i_text, max_length = 256)
            v_tokenized.append(line)

        padded = numpy.array([i + [1] * (max_len - len(i)) for i in v_tokenized])
        print('padded:', padded[0])
        print('len padded:', padded.shape)
        attention_mask = numpy.where(padded == 1, 0, 1)
        print('attention mask:', attention_mask[0])
        padded = torch.tensor(padded).to(torch.long)
        print("Padd = ",padded.size())
        attention_mask = torch.tensor(attention_mask)
        
        target = torch.tensor(label) 
        print(target)
        data_tensor = TensorDataset(padded, target) 
        train_loader = DataLoader(dataset = data_tensor, batch_size = 10)
        attention_mask = DataLoader(attention_mask, batch_size = 10)
        
        return train_loader, attention_mask
    
    def forward(self, input_ids, attention_mask):
        _, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
             return_dict=False
        )
#         self.drop = nn.Dropout(p=0.2)
#         output = self.drop(pooled_output)
        output = self.classifier(output)
        return output
#         return self.sigmoid(output)
    
    

In [28]:
import gc

def train_model(bert, save_path, df_train, df_val, num_epochs=1):
        text_train = df_train['Comment'].to_list()
        label_train = df_train['Rating'].to_list()
        text_test = df_test['Comment'].to_list()
        label_test = df_test['Rating'].to_list()
        
        Data_train, attention_train = bert.tokenize(text_train, label_train)
        Data_test, attention_test = bert.tokenize(text_test, label_test)
        
        dataloader_dict = {
            'train' : Data_train,
            'test' : Data_test
        }

        list_tmp = []
        
        for epoch in range(num_epochs + 1):
            if epoch == 0:
                print()
                continue
            print("Epoch {}/{} ".format(epoch, num_epochs))

            for phase in ['train', 'test']:
                if phase == 'train':
                    bert.train()
                    attention = attention_train
                else:
                    bert.eval()
                    attention = attention_test

                epoch_loss = 0.0
                epoch_corrects = 0

                for (inputs, labels), i_attention in tqdm(zip(dataloader_dict[phase], attention)):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    i_attention = i_attention.to(device)

                    bert.optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = bert(inputs,i_attention)

                        loss = bert.criterior(outputs, labels)

                        _, preds = torch.max(outputs, 1)


                        if phase == 'train':
                            loss.backward()
                            bert.optimizer.step()

                        epoch_loss += loss.item() * inputs.size(0)
                        epoch_corrects += torch.sum(preds == labels.data)

                    # with torch.no_grad():
                    #     outputs = bert(inputs, i_attention)
                    #     loss = bert.criterior(outputs, labels)
                    #
                    #     list_tmp.append(outputs)
                    #     print(outputs)
                    #
                    #     _, preds = torch.max(outputs, 1)
                    #
                    #     del outputs
                    #     torch.cuda.empty_cache()
                    #     gc.collect()
                    #
                    #     if phase == 'train':
                    #         loss.backward()
                    #         bert.optimizer.step()
                    #
                    #     epoch_loss += loss.item() * inputs.size(0)
                    #     epoch_corrects += torch.sum(preds == labels.data)


                epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
                epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)

                print("{} Loss: {:.4f}  Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
                torch.save(bert.bert.state_dict(), "./bert_model{}.pth".format(epoch))
                torch.save(bert.state_dict(), "./full_bert_model{}.pth".format(epoch))

In [29]:
# !pip install pytorch-pretrained-bert

In [30]:
# from pytorch_pretrained_bert.optimization import BertAdam

b =  Bert_fine_tuning()
train_model(b,"./", df_train, df_test, num_epochs=5)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples t

padded: [   0 1157 3589  164  133  123  390 7784  281 2273    2    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    

822it [03:56,  3.48it/s]


train Loss: 0.3377  Acc: 0.8795


100it [00:09, 10.78it/s]


...

test Loss: 0.2678  Acc: 0.9090
Epoch 5/5 


822it [03:55,  3.49it/s]


train Loss: 0.1123  Acc: 0.9642


100it [00:09, 10.82it/s]


test Loss: 0.2655  Acc: 0.9180


In [31]:
b = Bert_fine_tuning()
w = torch.load('full_bert_model1.pth')
b.load_state_dict(w)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<All keys matched successfully>

In [32]:
# predict

text_test = df_test['Comment'].to_list()
label_test = df_test['Rating'].to_list()

epoch_loss = 0.0
epoch_corrects = 0
net = b
net.eval()
Data_test, attention_test = b.tokenize(text_test, label_test)
attention = attention_test

list_pred = np.array([])
            
for (inputs, labels), i_attention in tqdm(zip(Data_test, attention)):
    inputs = inputs.to(device)
    labels = labels.to(device)
    i_attention = i_attention.to(device)
    
    outputs = b(inputs,i_attention)
        
    loss = b.criterior(outputs,labels)      
        
    _, preds = torch.max(outputs, 1)
    
#     print(preds.cpu().numpy())
    list_pred = np.append(list_pred,preds.cpu().numpy())
        
    epoch_loss += loss.item() * inputs.size(0)
    epoch_corrects += torch.sum(preds == labels.data)
            
epoch_loss = epoch_loss / len(Data_test.dataset)
epoch_acc = epoch_corrects.double() / len(Data_test.dataset)
            
print(" Loss: {:.4f}  Acc: {:.4f}".format( epoch_loss, epoch_acc))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


padded: [    0  4036   364  2977 13740    51  3585    17  1019  1325   246    15
   133   123 15370  2263   889  3619  1947    95  2607    51   203  5634
    17  2023   566 19505   889 10831  1947    95  2607    51    74    24
   137    76   107   566 19505   308     6    17  1834 12291  4481  1917
  1947    95    73    54  1236  6625  2008  8813     6    17  1019  1834
  1695    14  8861  2114   437    73  1834    68 49833  3030  2857  2537
     6  2122  4407    51   139  1498   131    17  1026 16767     7   946
  2621   234    51  2475    15  3538    54    44    17   549  2169     8
   281  2804    90   523 37973   167    28   946    36  3787  1595  1521
     2     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1 

100it [00:09, 11.00it/s]

 Loss: 0.2701  Acc: 0.8930


In [33]:
# torch.save(b.bert.state_dict(),"./bert_model.pth")

In [34]:
# list_pred

print("BERT Accuracy Score -> ",accuracy_score(list_pred, label_test)*100)

print(classification_report(label_test,list_pred))

BERT Accuracy Score ->  89.3
              precision    recall  f1-score   support

           0       0.68      0.91      0.78       204
           1       0.98      0.89      0.93       796

    accuracy                           0.89      1000
   macro avg       0.83      0.90      0.85      1000
weighted avg       0.91      0.89      0.90      1000



In [35]:
# check 1

# optimizer_grouped_parameters = [
#     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]
# num_train_optimization_steps = int(args.epochs*len(train_df)/args.batch_size/args.accumulation_steps)
# optimizer = AdamW(optimizer_grouped_parameters, lr=args.lr, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False

In [36]:
# Attention Layer
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self, W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'W_regularizer': self.W_regularizer,
            'u_regularizer': self.u_regularizer,
            'b_regularizer': self.b_regularizer,
            'W_constraint': self.W_constraint,
            'u_constraint': self.u_constraint,
            'b_constraint': self.b_constraint,
            'bias': self.bias
        })
        return config

def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

## combine with tree-model

In [37]:
def make_bert_features(v_text):
    global phobert, sw
    v_tokenized = []
    
    max_len = 256 
    for i_text in v_text:
#         i_text = standardize_data(i_text)
        line = tokenizer.encode(i_text, max_length = 256)
        v_tokenized.append(line)

    
    # Chèn thêm số 1 vào cuối câu nếu như không đủ 256 từ
    padded = numpy.array([i + [1] * (max_len - len(i)) for i in v_tokenized])
    print('padded:', padded[0])
    print('len padded:', padded.shape)

    # Đánh dấu các từ thêm vào = 0 để không tính vào quá trình lấy features
    attention_mask = numpy.where(padded == 1, 0, 1)
    print('attention mask:', attention_mask[0])

    padded = torch.tensor(padded).to(torch.long)
    print("Padd = ",padded.size())
    attention_mask = torch.tensor(attention_mask)
    
    padded = DataLoader(padded, batch_size=32)
    attention_mask = DataLoader(attention_mask, batch_size = 32)
    check = False
    v_feats = None
    for data, i_mask in zip(padded, attention_mask):
        with torch.no_grad():
            last_hidden_states = phobert(input_ids=data.to(device), attention_mask = i_mask.to(device))
            v_features = last_hidden_states[0][:, 0, :].cpu().numpy()
            if check is False: 
                v_feats = v_features
                check = True
            else:
                v_feats = np.append(v_feats, v_features, axis = 0)
                
        print(v_feats.shape)
                                    
    print(v_feats.shape)
    return v_feats

In [38]:


import numpy as np
tokenizer = v_tokenizer
phobert = b.bert


text_train = df_train['Comment'].to_list()
label_train = df_train['Rating'].to_list()
text_test = df_test['Comment'].to_list()
label_test = df_test['Rating'].to_list()



features_train = make_bert_features(text_train)
features_test = make_bert_features(text_test)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


padded: [   0 1157 3589  164  133  123  390 7784  281 2273    2    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    1
    1    1    1    1    1    1    1    1    1    1    1    1    1    

## BERT Keras

In [39]:
# def create_bert():
#
#     model = Sequential()
#     model.add(embed)
#     model.add(Dropout(0.2))
#     model.add(Dense(768, use_bias=True))
#     model.add(BatchNormalizationV1(500, epsilon=1e-5, momentum=0.1))
#     model.add(ReLU())
#     model.add(Dropout(0.1))
#     model.add(Dense(500))
#     #model.add(nn.Linear(in_features=768, out_features = 500, bias=True))
#     # model.add(nn.BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True).to(device))
#     # model.add(nn.ReLU(inplace=True))
#     # model.add(nn.Dropout(p=0.1))
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#     model.build(input_shape=(1,1,256,768))
#     model.summary()
#
#     return model

In [40]:
# bert = create_bert()
# bert.built()
# bert.fit(features_train, np.array(label_train))
# predictions_bert = bert.predict(features_test)
#
# print("BERT Accuracy Score -> ",accuracy_score(predictions_bert, label_test)*100)
# print(classification_report(label_test,predictions_bert))

In [41]:
# from sklearn.metrics import roc_auc_score
#
# step_factor = 0.02
# threshold_value = 0.2
# roc_score=0
# predicted_proba = bert.predict_proba(features_test) #probability of prediction
# while threshold_value <=0.9: #continue to check best threshold upto probability 0.8
#     temp_thresh = threshold_value
#     predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
#     print('Threshold',temp_thresh,'--',roc_auc_score(label_test, predicted))
#     if roc_score<roc_auc_score(label_test, predicted): #store the threshold for best classification
#         roc_score = roc_auc_score(label_test, predicted)
#         thrsh_score = threshold_value
#     threshold_value = threshold_value + step_factor
# print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)

## LIGBM

In [42]:
import lightgbm as lgb

lgbm = LGBMClassifier()
train = lgbm.fit(features_train,label_train)
predictions_lgbm = lgbm.predict(features_test)
# Use accuracy_score function to get the accuracy
print("ligbm Accuracy Score -> ",accuracy_score(predictions_lgbm, label_test)*100)
print(classification_report(label_test,predictions_lgbm))

ligbm Accuracy Score ->  94.1
              precision    recall  f1-score   support

           0       0.88      0.82      0.85       204
           1       0.96      0.97      0.96       796

    accuracy                           0.94      1000
   macro avg       0.92      0.90      0.91      1000
weighted avg       0.94      0.94      0.94      1000



In [43]:
from sklearn.metrics import roc_auc_score
predictions_lgbm = (lgbm.predict_proba(features_test)[:,1] >= 0.5).astype(bool)

In [44]:
print("ligbm Accuracy Score -> ",accuracy_score(predictions_lgbm, label_test)*100)
print(classification_report(label_test,predictions_lgbm))
print(roc_auc_score(label_test, predictions_lgbm))

ligbm Accuracy Score ->  94.1
              precision    recall  f1-score   support

           0       0.88      0.82      0.85       204
           1       0.96      0.97      0.96       796

    accuracy                           0.94      1000
   macro avg       0.92      0.90      0.91      1000
weighted avg       0.94      0.94      0.94      1000

0.8973174697014484


In [45]:
# from imblearn.ensemble import BalancedBaggingClassifier

# # boosting_type = 'goss', learning_rate = 0.05, n_estimators = 200,num_leaves = 20, scale_pos_weight = 0.7
# classifier = BalancedBaggingClassifier(base_estimator=LGBMClassifier(),
#                                 sampling_strategy='not majority',
#                                 replacement=False,
#                                 random_state=42)

# classifier.fit(features_train,label_train)
# predictions_bag = classifier.predict(features_test)
# # Use accuracy_score function to get the accuracy
# print("ligbm Accuracy Score -> ",accuracy_score(predictions_bag, label_test)*100)
# print(classification_report(label_test,predictions_bag))


In [46]:
from sklearn.metrics import roc_auc_score

step_factor = 0.02 
threshold_value = 0.2 
roc_score=0
predicted_proba = lgbm.predict_proba(features_test) #probability of prediction
while threshold_value <=0.9: #continue to check best threshold upto probability 0.8
    temp_thresh = threshold_value
    predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
    print('Threshold',temp_thresh,'--',roc_auc_score(label_test, predicted))
    if roc_score<roc_auc_score(label_test, predicted): #store the threshold for best classification
        roc_score = roc_auc_score(label_test, predicted)
        thrsh_score = threshold_value
    threshold_value = threshold_value + step_factor
print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)

Threshold 0.2 -- 0.877771208986107
Threshold 0.22 -- 0.8795940486747462
Threshold 0.24 -- 0.8789659079712288
Threshold 0.26 -- 0.8789659079712288
Threshold 0.28 -- 0.8789659079712288
Threshold 0.30000000000000004 -- 0.8838678687555426
Threshold 0.32000000000000006 -- 0.8887698295398563
Threshold 0.3400000000000001 -- 0.8936717903241698
Threshold 0.3600000000000001 -- 0.8954946300128092
Threshold 0.3800000000000001 -- 0.8954946300128092
Threshold 0.40000000000000013 -- 0.8954946300128092
Threshold 0.42000000000000015 -- 0.8954946300128092
Threshold 0.44000000000000017 -- 0.8948664893092916
Threshold 0.4600000000000002 -- 0.8948664893092916
Threshold 0.4800000000000002 -- 0.8973174697014484
Threshold 0.5000000000000002 -- 0.8973174697014484
Threshold 0.5200000000000002 -- 0.8997684500936053
Threshold 0.5400000000000003 -- 0.9040422701744014
Threshold 0.5600000000000003 -- 0.9058651098630408
Threshold 0.5800000000000003 -- 0.90768794955168
Threshold 0.6000000000000003 -- 0.90768794955168


In [47]:
predictions_lgbm = lgbm.predict(features_train)
# Use accuracy_score function to get the accuracy
print("ligbm Accuracy Score -> ",accuracy_score(predictions_lgbm, label_train)*100)

ligbm Accuracy Score ->  99.97565429093123


## ADA boost

In [48]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier()
clf.fit(features_train,label_train)

predictions_ada = clf.predict(features_test)
# Use accuracy_score function to get the accuracy
print("Ada Accuracy Score -> ",accuracy_score(predictions_ada, label_test)*100)
print(classification_report(label_test,predictions_ada))

joblib.dump(clf, "ada.pkl")

Ada Accuracy Score ->  93.2
              precision    recall  f1-score   support

           0       0.84      0.82      0.83       204
           1       0.95      0.96      0.96       796

    accuracy                           0.93      1000
   macro avg       0.90      0.89      0.89      1000
weighted avg       0.93      0.93      0.93      1000



['ada.pkl']

## Random forest

In [49]:
rf = RandomForestClassifier()
rf.fit(features_train,label_train)
predictions_rf = rf.predict(features_test)
# Use accuracy_score function to get the accuracy
print("rf Accuracy Score -> ",accuracy_score(predictions_rf, label_test)*100)
print(classification_report(label_test,predictions_rf))

joblib.dump(rf, "rf.pkl")

rf Accuracy Score ->  94.0
              precision    recall  f1-score   support

           0       0.87      0.83      0.85       204
           1       0.96      0.97      0.96       796

    accuracy                           0.94      1000
   macro avg       0.91      0.90      0.91      1000
weighted avg       0.94      0.94      0.94      1000



['rf.pkl']

In [50]:
predictions_rf = rf.predict(features_test)
# Use accuracy_score function to get the accuracy
print("rf Accuracy Score -> ",accuracy_score(predictions_rf, label_test)*100)
print(classification_report(label_test,predictions_rf))

rf Accuracy Score ->  94.0
              precision    recall  f1-score   support

           0       0.87      0.83      0.85       204
           1       0.96      0.97      0.96       796

    accuracy                           0.94      1000
   macro avg       0.91      0.90      0.91      1000
weighted avg       0.94      0.94      0.94      1000



In [51]:
step_factor = 0.02 
threshold_value = 0.2 
roc_score=0
predicted_proba = rf.predict_proba(features_test) #probability of prediction
while threshold_value <=0.9: #continue to check best threshold upto probability 0.8
    temp_thresh = threshold_value
    predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
    print('Threshold',temp_thresh,'--',roc_auc_score(label_test, predicted))
    if roc_score<roc_auc_score(label_test, predicted): #store the threshold for best classification
        roc_score = roc_auc_score(label_test, predicted)
        thrsh_score = threshold_value
    threshold_value = threshold_value + step_factor
print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)

Threshold 0.2 -- 0.8454650704502906
Threshold 0.22 -- 0.8454650704502906
Threshold 0.24 -- 0.8491107498275692
Threshold 0.26 -- 0.8552074095970046
Threshold 0.28 -- 0.8619322100699576
Threshold 0.30000000000000004 -- 0.8619322100699576
Threshold 0.32000000000000006 -- 0.8637550497585968
Threshold 0.3400000000000001 -- 0.8667725884323578
Threshold 0.3600000000000001 -- 0.8802221893782639
Threshold 0.3800000000000001 -- 0.8851241501625777
Threshold 0.40000000000000013 -- 0.8844960094590602
Threshold 0.42000000000000015 -- 0.8838678687555426
Threshold 0.44000000000000017 -- 0.8856907084441817
Threshold 0.4600000000000002 -- 0.8893363878214603
Threshold 0.4800000000000002 -- 0.8960611882944132
Threshold 0.5000000000000002 -- 0.8985121686865701
Threshold 0.5200000000000002 -- 0.9034141294708838
Threshold 0.5400000000000003 -- 0.9021578480638487
Threshold 0.5600000000000003 -- 0.9033525470489704
Threshold 0.5800000000000003 -- 0.9076263671297665
Threshold 0.6000000000000003 -- 0.910077347521

In [52]:
# df_test['predicted_label'] = predictions_rf
# df_test.to_csv('test_predict.csv', encoding = 'utf-8-sig')

In [53]:
# df_test

## XGBoost

In [54]:
import xgboost as xgb

xgb = xgb.XGBClassifier()
xgb.fit(features_train,label_train)
y_train_predict = xgb.predict(features_train)
print("XGB Accuracy Score of train set-> ",accuracy_score(label_train, y_train_predict)*100)

y_test_predict = xgb.predict(features_test)
print("xgb Accuracy Score -> ",accuracy_score(label_test,y_test_predict)*100)
print(classification_report(label_test,y_test_predict))

joblib.dump(xgb, "xgb.pkl")

XGB Accuracy Score of train set->  99.97565429093123
xgb Accuracy Score ->  94.1
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       204
           1       0.96      0.97      0.96       796

    accuracy                           0.94      1000
   macro avg       0.91      0.90      0.91      1000
weighted avg       0.94      0.94      0.94      1000



['xgb.pkl']

In [55]:
print("xgb Accuracy Score -> ",accuracy_score(label_test,y_test_predict)*100)

xgb Accuracy Score ->  94.1


## CAT BOOST

In [56]:
# from catboost import CatBoostClassifier
# clf = CatBoostClassifier(task_type = "GPU", iterations = 1100,depth = 15)
#
# # ,iterations=1000,learning_rate = 0.15,depth =10)
# # , ,cat_features = LIST_FEAT1 + LIST_FEAT2 + LIST_FEAT5 + LIST_FEAT6 + LIST_FEAT7 + LIST_FEAT8)
# clf.fit(features_train,label_train)
# y_train_predict = clf.predict(features_train)
# print("CBoost Accuracy Score of train set-> ",accuracy_score(label_train, y_train_predict)*100)
#
# y_test_predict = clf.predict(features_test)
# print(classification_report(label_test,y_test_predict))
#
# joblib.dump(clf, "catboost.pkl")

## SVM

In [57]:
SVM = SVC()
SVM.fit(features_train,label_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(features_test)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, label_test)*100)

joblib.dump(SVM, "svm.pkl")

SVM Accuracy Score ->  94.0


['svm.pkl']

## Logistic Regression

In [58]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression()

reg.fit(features_train,label_train)
y_train_predict = reg.predict(features_train)
print("LG Accuracy Score of train set-> ",accuracy_score(label_train, y_train_predict)*100)

y_test_predict = reg.predict(features_test)
print("LG Accuracy Score -> ",accuracy_score(label_test,y_test_predict)*100)
print(classification_report(label_test,y_test_predict))

joblib.dump(reg, "lg.pkl")

LG Accuracy Score of train set->  94.96043822276323
LG Accuracy Score ->  93.30000000000001
              precision    recall  f1-score   support

           0       0.84      0.83      0.83       204
           1       0.96      0.96      0.96       796

    accuracy                           0.93      1000
   macro avg       0.90      0.89      0.90      1000
weighted avg       0.93      0.93      0.93      1000



/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


['lg.pkl']

In [59]:
predictions_vt = (reg.predict_proba(features_test)[:,1] >= 0.51).astype(bool)

print("LG Accuracy Score -> ",accuracy_score(predictions_vt, label_test)*100)
print(classification_report(label_test,predictions_vt))
print(roc_auc_score(label_test, predictions_vt))

LG Accuracy Score ->  93.4
              precision    recall  f1-score   support

           0       0.84      0.83      0.84       204
           1       0.96      0.96      0.96       796

    accuracy                           0.93      1000
   macro avg       0.90      0.90      0.90      1000
weighted avg       0.93      0.93      0.93      1000

0.8965661641541038


In [60]:
step_factor = 0.02 
threshold_value = 0.2 
roc_score=0
predicted_proba = reg.predict_proba(features_test) #probability of prediction
while threshold_value <=0.8: #continue to check best threshold upto probability 0.8
    temp_thresh = threshold_value
    predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
    print('Threshold',temp_thresh,'--',roc_auc_score(label_test, predicted))
    if roc_score<roc_auc_score(label_test, predicted): #store the threshold for best classification
        roc_score = roc_auc_score(label_test, predicted)
        thrsh_score = threshold_value
    threshold_value = threshold_value + step_factor
print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)

Threshold 0.2 -- 0.8748152527342594
Threshold 0.22 -- 0.8760099517193811
Threshold 0.24 -- 0.8778327914080204
Threshold 0.26 -- 0.8790274903931421
Threshold 0.28 -- 0.8808503300817814
Threshold 0.30000000000000004 -- 0.8814168883633855
Threshold 0.32000000000000006 -- 0.8863188491476993
Threshold 0.3400000000000001 -- 0.8856907084441817
Threshold 0.3600000000000001 -- 0.8881416888363386
Threshold 0.3800000000000001 -- 0.8899645285249779
Threshold 0.40000000000000013 -- 0.8911592275100996
Threshold 0.42000000000000015 -- 0.8923539264952213
Threshold 0.44000000000000017 -- 0.8935486254803429
Threshold 0.4600000000000002 -- 0.8929204847768252
Threshold 0.4800000000000002 -- 0.8953714651689821
Threshold 0.5000000000000002 -- 0.894115183761947
Threshold 0.5200000000000002 -- 0.8965661641541038
Threshold 0.5400000000000003 -- 0.8940536013400335
Threshold 0.5600000000000003 -- 0.8965045817321904
Threshold 0.5800000000000003 -- 0.9026012415016257
Threshold 0.6000000000000003 -- 0.9068750615824

## MLP Classifier

In [61]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(activation = "logistic", alpha = 0.1, hidden_layer_sizes = (10, 10, 10),
                            learning_rate = "constant", max_iter = 300, random_state = 42)

mlp.fit(features_train,label_train)
y_train_predict = mlp.predict(features_train)
print("MLP Accuracy Score of train set-> ",accuracy_score(label_train, y_train_predict)*100)

y_test_predict = mlp.predict(features_test)
print("MLP Accuracy Score -> ",accuracy_score(label_test,y_test_predict)*100)
print(classification_report(label_test,y_test_predict))

joblib.dump(reg, "mlp.pkl")

MLP Accuracy Score of train set->  94.71698113207547
MLP Accuracy Score ->  94.0
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       204
           1       0.96      0.96      0.96       796

    accuracy                           0.94      1000
   macro avg       0.91      0.90      0.91      1000
weighted avg       0.94      0.94      0.94      1000



['mlp.pkl']

## NuSVC

In [62]:
from sklearn.svm import NuSVC

nu_svc = NuSVC(degree = 1, kernel = "linear", nu = 0.1, probability = True, max_iter=300)

nu_svc.fit(features_train,label_train)
y_train_predict = nu_svc.predict(features_train)
print("NuSVC Accuracy Score of train set-> ",accuracy_score(label_train, y_train_predict)*100)

y_test_predict = nu_svc.predict(features_test)
print("NuSVC Accuracy Score -> ",accuracy_score(label_test,y_test_predict)*100)
print(classification_report(label_test,y_test_predict))

joblib.dump(reg, "nusvc.pkl")

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=300).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


NuSVC Accuracy Score of train set->  92.87888009738283
NuSVC Accuracy Score ->  94.69999999999999
              precision    recall  f1-score   support

           0       0.88      0.85      0.87       204
           1       0.96      0.97      0.97       796

    accuracy                           0.95      1000
   macro avg       0.92      0.91      0.92      1000
weighted avg       0.95      0.95      0.95      1000



['nusvc.pkl']

## LSTM

In [63]:
#Building the model
def create_model():
    model = Sequential()
    embed = Embedding(input_dim = 60000, output_dim = 20, input_length = features_train.shape[1])
    model.add(embed)
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(200, return_sequences = True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(200, return_sequences = True)))
    model.add(AttentionWithContext())
    model.add(Dropout(0.1))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.summary()
    return model

## Voting

In [64]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from keras.models import load_model

rf = RandomForestClassifier(n_estimators=500,min_samples_leaf=2,
                                min_samples_split= 10, max_features = 'sqrt', criterion = 'entropy', bootstrap= True,
                              random_state=42)
lgbm = LGBMClassifier(boosting_type = 'goss', learning_rate = 0.05, n_estimators = 200,num_leaves = 20, scale_pos_weight = 0.7)
lg = LogisticRegression(max_iter=200)
xgb = XGBClassifier(n_estimators = 200, max_depth=20,tree_method="hist")
svm = SVC(kernel='linear', probability=True, gamma=0.2, degree=1, verbose=True, tol=1e-5)
ada = AdaBoostClassifier(n_estimators=200, random_state=1, learning_rate=1e-5)
mlp = MLPClassifier(activation = "relu", alpha = 0.1, hidden_layer_sizes = (10,10,10),
                            learning_rate = "constant", max_iter = 2000, random_state = 1000)
nu_svc = NuSVC(degree = 1, kernel = "rbf", nu = 0.25, probability = True)

rf = joblib.load("rf.pkl")
lg = joblib.load("lg.pkl")
xgb = joblib.load("xgb.pkl")
svm = joblib.load("svm.pkl")
ada = joblib.load("ada.pkl")
mlp = joblib.load("mlp.pkl")
nu_svc = joblib.load("nusvc.pkl")

lstm = KerasClassifier(build_fn= lambda: create_model())
lstm._estimator_type = "classifier"

# bert = KerasClassifier(build_fn = lambda: create_bert())
# bert._estimator_type = 'classifier'

estimator = []
# estimator.append(('LR', 
#                   LogisticRegression(solver ='lbfgs', 
#                                      multi_class ='multinomial', 
#                                      max_iter = 200)))

estimator.append(('RF', rf))
estimator.append(('XGB', xgb))
estimator.append(('LGBM', lgbm))
# estimator.append(('MLP', mlp))
# estimator.append(('SVM', svm ))
# estimator.append(('NuSVC', nu_svc))
estimator.append(('LG', lg))
# estimator.append(('LSTM', lstm))
# estimator.append(('ADA', ada))
# estimator.append(('BERT', bert))
# estimator.append(('LSTM', lstm))

# voting ='soft'
vot_hard = VotingClassifier(estimators = estimator, voting ='soft')
vot_hard.fit(features_train,label_train)
y_pred = vot_hard.predict(features_test)
                 
print("Voting Accuracy Score -> ",accuracy_score(y_pred, label_test)*100)

print(classification_report(label_test,y_pred))

Voting Accuracy Score ->  93.89999999999999
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       204
           1       0.96      0.96      0.96       796

    accuracy                           0.94      1000
   macro avg       0.91      0.90      0.91      1000
weighted avg       0.94      0.94      0.94      1000



/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [65]:
predictions_vt = (vot_hard.predict_proba(features_test)[:,1] >= 0.47).astype(bool)

print("voting Accuracy Score -> ",accuracy_score(predictions_vt, label_test)*100)
print(classification_report(label_test,predictions_vt))
print(roc_auc_score(label_test, predictions_vt))

voting Accuracy Score ->  93.8
              precision    recall  f1-score   support

           0       0.86      0.83      0.85       204
           1       0.96      0.96      0.96       796

    accuracy                           0.94      1000
   macro avg       0.91      0.90      0.90      1000
weighted avg       0.94      0.94      0.94      1000

0.8990787269681743


In [66]:
step_factor = 0.02 
threshold_value = 0.2 
roc_score=0
predicted_proba = vot_hard.predict_proba(features_test) #probability of prediction
while threshold_value <=0.9: #continue to check best threshold upto probability 0.8
    temp_thresh = threshold_value
    predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
    print('Threshold',temp_thresh,'--',roc_auc_score(label_test, predicted))
    if roc_score<roc_auc_score(label_test, predicted): #store the threshold for best classification
        roc_score = roc_auc_score(label_test, predicted)
        thrsh_score = threshold_value
    threshold_value = threshold_value + step_factor
print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)

Threshold 0.2 -- 0.8723026899201891
Threshold 0.22 -- 0.8723026899201891
Threshold 0.24 -- 0.8808503300817814
Threshold 0.26 -- 0.8826731697704208
Threshold 0.28 -- 0.8869469898512169
Threshold 0.30000000000000004 -- 0.8863188491476993
Threshold 0.32000000000000006 -- 0.8850625677406642
Threshold 0.3400000000000001 -- 0.8899645285249779
Threshold 0.3600000000000001 -- 0.8899645285249779
Threshold 0.3800000000000001 -- 0.8893363878214603
Threshold 0.40000000000000013 -- 0.8887082471179427
Threshold 0.42000000000000015 -- 0.8985121686865701
Threshold 0.44000000000000017 -- 0.9009631490787271
Threshold 0.4600000000000002 -- 0.8997068676716918
Threshold 0.4800000000000002 -- 0.8990787269681743
Threshold 0.5000000000000002 -- 0.9015297073603311
Threshold 0.5200000000000002 -- 0.9033525470489704
Threshold 0.5400000000000003 -- 0.9058035274411271
Threshold 0.5600000000000003 -- 0.9058035274411271
Threshold 0.5800000000000003 -- 0.908254507833284
Threshold 0.6000000000000003 -- 0.9107054882254

## Stack

In [67]:
from sklearn.ensemble import StackingClassifier

estimators = [
    # ('RF', rf),
    # ('XGB', xgb),
    # ('LGBM', lgbm),
    # ('SVM', svm),
    ("RF", rf),
    ('XGB', xgb),
    ('LGBM', lgbm),
    ('LSTM', lstm),
    # ('NuSVC', nu_svc)
]

st = StackingClassifier(estimators=estimators, final_estimator = ada)
st.fit(features_train,label_train)
y_pred = st.predict(features_test)
                 
print("ST Accuracy Score -> ",accuracy_score(y_pred, label_test)*100)

print(classification_report(label_test,y_pred))

2022-12-12 18:03:55.331213: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 18:03:55.333215: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 18:03:55.333776: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-12 18:03:55.334534: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 768, 20)           1200000   
_________________________________________________________________
dropout (Dropout)            (None, 768, 20)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 768, 400)          353600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 768, 400)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 768, 400)          961600    
_________________________________________________________________
attention_with_context (Atte (None, 400)               160800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 400)               0

2022-12-12 18:03:58.219180: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 25236480 exceeds 10% of free system memory.
2022-12-12 18:03:58.255137: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 25236480 exceeds 10% of free system memory.
2022-12-12 18:03:58.281868: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-12 18:04:04.263998: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


257/257 [==============================] - 60s 207ms/step - loss: 0.6388 - accuracy: 0.6693
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 768, 20)           1200000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 768, 20)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 768, 400)          353600    
_________________________________________________________________
dropout_4 (Dropout)          (None, 768, 400)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 768, 400)          961600    
_________________________________________________________________
attention_with_context_1 (At (None, 400)               160800    
____________________________

2022-12-12 18:14:40.078600: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 20189184 exceeds 10% of free system memory.


206/206 [==============================] - 48s 208ms/step - loss: 0.6414 - accuracy: 0.6677
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 768, 20)           1200000   
_________________________________________________________________
dropout_6 (Dropout)          (None, 768, 20)           0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 768, 400)          353600    
_________________________________________________________________
dropout_7 (Dropout)          (None, 768, 400)          0         
_________________________________________________________________
bidirectional_5 (Bidirection (None, 768, 400)          961600    
_________________________________________________________________
attention_with_context_2 (At (None, 400)               160800    
____________________________

2022-12-12 18:16:12.980646: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 20189184 exceeds 10% of free system memory.


206/206 [==============================] - 48s 206ms/step - loss: 0.6390 - accuracy: 0.6681
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 768, 20)           1200000   
_________________________________________________________________
dropout_9 (Dropout)          (None, 768, 20)           0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 768, 400)          353600    
_________________________________________________________________
dropout_10 (Dropout)         (None, 768, 400)          0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 768, 400)          961600    
_________________________________________________________________
attention_with_context_3 (At (None, 400)               160800    
____________________________

2022-12-12 18:17:46.997451: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 20189184 exceeds 10% of free system memory.


206/206 [==============================] - 48s 208ms/step - loss: 0.6397 - accuracy: 0.6681
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 768, 20)           1200000   
_________________________________________________________________
dropout_12 (Dropout)         (None, 768, 20)           0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 768, 400)          353600    
_________________________________________________________________
dropout_13 (Dropout)         (None, 768, 400)          0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 768, 400)          961600    
_________________________________________________________________
attention_with_context_4 (At (None, 400)               160800    
____________________________

In [68]:
predictions_st = (st.predict_proba(features_test)[:,1] >= 0.5).astype(bool)

print("ST Accuracy Score -> ",accuracy_score(predictions_st, label_test)*100)
print(classification_report(label_test,predictions_st))
print(roc_auc_score(label_test, predictions_st))

ST Accuracy Score ->  93.89999999999999
              precision    recall  f1-score   support

           0       0.87      0.82      0.85       204
           1       0.95      0.97      0.96       796

    accuracy                           0.94      1000
   macro avg       0.91      0.89      0.90      1000
weighted avg       0.94      0.94      0.94      1000

0.894238348605774


In [69]:
step_factor = 0.02 
threshold_value = 0.2 
roc_score=0
predicted_proba = st.predict_proba(features_test) #probability of prediction
while threshold_value <=0.8: #continue to check best threshold upto probability 0.8
    temp_thresh = threshold_value
    predicted = (predicted_proba [:,1] >= temp_thresh).astype('int') #change the class boundary for prediction
    print('Threshold',temp_thresh,'--',roc_auc_score(label_test, predicted))
    if roc_score<roc_auc_score(label_test, predicted): #store the threshold for best classification
        roc_score = roc_auc_score(label_test, predicted)
        thrsh_score = threshold_value
    threshold_value = threshold_value + step_factor
print('---Optimum Threshold ---',thrsh_score,'--ROC--',roc_score)

Threshold 0.2 -- 0.5
Threshold 0.22 -- 0.5
Threshold 0.24 -- 0.5
Threshold 0.26 -- 0.5
Threshold 0.28 -- 0.5
Threshold 0.30000000000000004 -- 0.5
Threshold 0.32000000000000006 -- 0.5
Threshold 0.3400000000000001 -- 0.5
Threshold 0.3600000000000001 -- 0.5
Threshold 0.3800000000000001 -- 0.5581953887082471
Threshold 0.40000000000000013 -- 0.5704502906690315
Threshold 0.42000000000000015 -- 0.5704502906690315
Threshold 0.44000000000000017 -- 0.5704502906690315
Threshold 0.4600000000000002 -- 0.5704502906690315
Threshold 0.4800000000000002 -- 0.6739580254212238
Threshold 0.5000000000000002 -- 0.894238348605774
Threshold 0.5200000000000002 -- 0.8193294905902059
Threshold 0.5400000000000003 -- 0.5396344467435216
Threshold 0.5600000000000003 -- 0.5396344467435216
Threshold 0.5800000000000003 -- 0.5396344467435216
Threshold 0.6000000000000003 -- 0.5396344467435216
Threshold 0.6200000000000003 -- 0.5396344467435216
Threshold 0.6400000000000003 -- 0.5396344467435216
Threshold 0.6600000000000004 

In [70]:
b

Bert_fine_tuning(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((76

In [71]:
df_submit = pd.read_csv('/kaggle/input/mlkaggle/vuong_test.csv')
select_indices = list(np.where(df_submit.loc[:,"Comment"].isnull()))
for item in select_indices[0]: #select_indices is an array, you need to access the first element to print the indexes
    print(item)
df_submit = df_submit.dropna()
df.shape

(9069, 4)

In [72]:
text_sub = df_submit['Comment'].to_list()
features_sub = make_bert_features(text_sub)

padded: [    0  2272  4451  6639  1947 34106 21834 21913  6620  1947    58    25
    97  1006    51   133  1329 11206  3628  1152   946   475    15    36
   946    85  4578  1006    66    10    76    18   533  2905    58  1329
    45   700  8210  2169     8  1236  8017  1595    25   189    85    54
    68    17    55  1279    51    25  1005 14307  4453    54  5634   204
  3628  3787  1595  1521  5119  3787  1595  1521  5119     2     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1 

In [73]:
predicted_sub = vot_hard.predict_proba(features_sub)

In [74]:
df_submit['Rating'] = predicted_sub[:,1]

In [75]:
df_submit['Comment'].iloc[0]

'trà táo 35k cookie socola 38k nước ở đây bình_thường nhưng giá hơi chát ạ không_gian quán giống với nhiều quán khác phục bình_thường chỉ có 2 người phục_vụ đợi nước hơi bị lâu í nói_chung là delio ở nơi khác thì mình không biết thế_nào nhưng ở hồ đắc di thì nhạt quá ạ emo khóc ròng emo khóc ròng '

In [76]:
df_submit[['RevId','Rating']].to_csv('submit_bert.csv', index=False)

In [77]:
# predict


net = b
net.eval()
Data_test, attention_test = b.tokenize(text_test, label_test)
attention = attention_test

list_pred = np.array([])
            
for (inputs, labels), i_attention in tqdm(zip(Data_test, attention)):
    inputs = inputs.to(device)
    labels = labels.to(device)
    i_attention = i_attention.to(device)
    
    outputs = b(inputs,i_attention)
        
    loss = b.criterior(outputs,labels)      
        
    _, preds = torch.max(outputs, 1)
    
#     print(preds.cpu().numpy())
    list_pred = np.append(list_pred,preds.cpu().numpy())
        
    epoch_loss += loss.item() * inputs.size(0)
    epoch_corrects += torch.sum(preds == labels.data)
            
epoch_loss = epoch_loss / len(Data_test.dataset)
epoch_acc = epoch_corrects.double() / len(Data_test.dataset)
            
print(" Loss: {:.4f}  Acc: {:.4f}".format( epoch_loss, epoch_acc))

padded: [    0  4036   364  2977 13740    51  3585    17  1019  1325   246    15
   133   123 15370  2263   889  3619  1947    95  2607    51   203  5634
    17  2023   566 19505   889 10831  1947    95  2607    51    74    24
   137    76   107   566 19505   308     6    17  1834 12291  4481  1917
  1947    95    73    54  1236  6625  2008  8813     6    17  1019  1834
  1695    14  8861  2114   437    73  1834    68 49833  3030  2857  2537
     6  2122  4407    51   139  1498   131    17  1026 16767     7   946
  2621   234    51  2475    15  3538    54    44    17   549  2169     8
   281  2804    90   523 37973   167    28   946    36  3787  1595  1521
     2     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1     1     1     1     1     1
     1     1     1     1     1     1     1 

100it [00:09, 11.01it/s]

 Loss: 0.2703  Acc: 1.7860
